In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, date

pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 8)
pd.set_option('display.max_rows', 15)
pd.set_option('display.width', 65)
%matplotlib inline

## Категориальные данные

Категориальная переменная – это тип переменной в статистике, который представляет собой ограниченный и часто фиксированный набор значений.

К распространенным
категориальным переменным относятся пол (когда есть два значения: мужчина
и женщина) или группа крови (например, A, B и 0).


#### Создание категориальной переменной из списка

In [2]:
lmh_values = ["low", "high", "medium", "medium", "high"]
lmh_cat = pd.Categorical(lmh_values)
lmh_cat

['low', 'high', 'medium', 'medium', 'high']
Categories (3, object): ['high', 'low', 'medium']

#### Смотрим категории

In [3]:
# Объект categories создает индекс, состоящий из трех различных значений в заданном списке.
lmh_cat.categories

Index(['high', 'low', 'medium'], dtype='object')

In [4]:
# Свойство .codes показывает коды (целочисленные значения)
# для каждого значения категориальной переменной
lmh_cat.codes


array([1, 0, 2, 2, 0], dtype=int8)

In [5]:
# Создадим объект категорий из списка, но при этом ЯВНО УКАЖЕМ КАТЕГОРИИ
# будет собдюден порядок указанных категорий
lmh_cat = pd.Categorical(lmh_values, categories=["low", "medium", "high"])
print(lmh_cat, end='\n\n')
print(lmh_cat.value_counts())


['low', 'high', 'medium', 'medium', 'high']
Categories (3, object): ['low', 'medium', 'high']

low       1
medium    2
high      2
Name: count, dtype: int64


#### Коды выглядят так

In [6]:
lmh_cat.codes

array([0, 2, 1, 1, 2], dtype=int8)

#### Сортировка выполняется с помощью кодов, лежащих в основе каждого значения

In [7]:
lmh_cat.sort_values()

['low', 'medium', 'medium', 'high', 'high']
Categories (3, object): ['low', 'medium', 'high']

#### Кроме того категориальную переменню можно создать В ВИДЕ СЕРИИ у которой задать тип данный (dtype) в виде - category

In [8]:
cat_series = pd.Series(lmh_values, dtype='category')
cat = ["low", "medium", "low", "high"]
len_cat = len(cat)
val = [i for i in np.random.random(len_cat)]
lmh_values

df_cat = pd.DataFrame({
    'category': cat,
    'value': [cat[i] for i in np.random.randint(0, len_cat-1, len_cat)]
})
new_df_cat = df_cat.set_index('category')

new_col = pd.Series([i for i in np.random.random(len_cat)], index=new_df_cat.index)
new_df = pd.DataFrame({'value': val}, index=new_df_cat.index)

new_df_with_series = pd.concat([new_df_cat, new_col], axis=1)
new_df_with_dataframe = pd.concat([new_df_cat, new_df], axis=1)

# print(new_df_cat.loc['low'], end='\n\n')
print('new_df_cat', '\n', new_df_cat, '\n')
print('new_col', '\n', new_col, '\n')
print('new_df', '\n', new_df, '\n')
print(new_df_with_series, end='\n\n')
print(new_df_with_dataframe)

new_df_cat 
            value
category        
low          low
medium       low
low          low
high      medium 

new_col 
 category
low       0.644365
medium    0.368483
low       0.381953
high      0.960175
dtype: float64 

new_df 
              value
category          
low       0.222054
medium    0.176284
low       0.214941
high      0.568771 

           value         0
category                  
low          low  0.644365
medium       low  0.368483
low          low  0.381953
high      medium  0.960175

           value     value
category                  
low          low  0.222054
medium       low  0.176284
low          low  0.214941
high      medium  0.568771


#### Переименуем колонки, имеющие одно и тоже название

In [9]:
print(new_df_with_dataframe)
new_df_with_dataframe.columns = ['value', 'randomVal']
new_df_with_dataframe

           value     value
category                  
low          low  0.222054
medium       low  0.176284
low          low  0.214941
high      medium  0.568771


           value  randomVal
category                   
low          low   0.222054
medium       low   0.176284
low          low   0.214941
high      medium   0.568771

In [10]:
# Еще один способ создания категориальной переменной состоит в том, чтобы сначала создать
# объект Series а потом преобразовать столбец с данными в категориальную переменную,
# используя метод .astype('category')

s = pd.Series(lmh_values)
print(s)
as_cat = s.astype('category')
# добавим строчку middle к серии s
print(pd.concat([as_cat, pd.Series(['middle'])]))
as_cat, as_cat.cat.categories

0       low
1      high
2    medium
3    medium
4      high
dtype: object
0       low
1      high
2    medium
3    medium
4      high
0    middle
dtype: object


(0       low
 1      high
 2    medium
 3    medium
 4      high
 dtype: category
 Categories (3, object): ['high', 'low', 'medium'],
 Index(['high', 'low', 'medium'], dtype='object'))

#### Получаем индекс категориальной переменной

In [11]:
# Серия, созданная как объект categorical имеет свойство .CAT, которое позволяет получить
# доступ к категориальной переменной.
as_cat.cat

#### Получаем коды категорий

In [12]:
as_cat.cat.categories

Index(['high', 'low', 'medium'], dtype='object')

In [13]:
# Создание датафрейма из 5 значений
np.random.seed(123456)
values = np.random.randint(0, 100, 5)
bins = pd.DataFrame({"Values": values})
bins

   Values
0      65
1      49
2      56
3      43
4      43

### Несколько пандасовский функций также возвращают объекты Categorical. 

**Одна из таких - pd.cut()**, которая создает группы наблюдений, расположенных внутри определенных диапазонов значений.


In [14]:
# Разбираем значения на 10 групп.
bins['Group'] = pd.cut(values, range(0, 101, 10))

# Столбец group представляет из себя категориальную переменню
bins.Group

0    (60, 70]
1    (40, 50]
2    (50, 60]
3    (40, 50]
4    (40, 50]
Name: Group, dtype: category
Categories (10, interval[int64, right]): [(0, 10] < (10, 20] < (20, 30] < (30, 40] ... (60, 70] < (70, 80] < (80, 90] < (90, 100]]

In [15]:
bins.Group.cat.codes

0    6
1    4
2    5
3    4
4    4
dtype: int8

In [16]:
bins.Group.cat.categories

IntervalIndex([  (0, 10],  (10, 20],  (20, 30],  (30, 40],
                (40, 50],  (50, 60],  (60, 70],  (70, 80],
                (80, 90], (90, 100]],
              dtype='interval[int64, right]')

#### Создадим УПОРЯДОЧЕННУЮ категореальную переменную из названий драгоценных металлов

ПОРЯДОК ВАЖЕН для определения относительной ценности металла.

In [17]:
metal_values = ["bronze", "gold", "silver", "bronze"]
metal_categories =  ["bronze", "silver", "gold"]
metals = pd.Categorical(metal_values, 
                        categories = metal_categories, 
                        ordered = True)

# Объект сатегории ИМЕЕТ СТРОГО УПОРЯДОЧЕННЫЕ категории
print(metals)


['bronze', 'gold', 'silver', 'bronze']
Categories (3, object): ['bronze' < 'silver' < 'gold']


#### Порядок категорий можно использовать для сортировки или сравнения значений одного объекта Categorical со значением др.

In [18]:
# Меняем значения
metals_reversed_values = pd.Categorical(
    metal_values[::-1],
    categories = metals.categories,
    ordered=True
)
metals_reversed_values

['bronze', 'silver', 'gold', 'bronze']
Categories (3, object): ['bronze' < 'silver' < 'gold']

#### Значения этих категориальных переменный metals и metals_reversed_values можно сравнить.

In [19]:
# Сравним значения 2 категориальных переменных. Библиотека пандас выполняет сравниение,
# сопоставляя коды значения.
print(metals <= metals_reversed_values)
print(metals.codes)
print(metals_reversed_values.codes)

[ True False  True  True]
[0 2 1 0]
[0 1 2 0]


In [20]:
# Создадим категориальную переменную со значением, которое нельзя отнести ни к одной из
# категорий, поэтому для него будет получено значене NaN

other_met = pd.Categorical(["bronze", "copper"], categories=metal_categories)
print(other_met)
print(other_met.categories)
print(other_met.codes)

['bronze', NaN]
Categories (3, object): ['bronze', 'silver', 'gold']
Index(['bronze', 'silver', 'gold'], dtype='object')
[ 0 -1]


### Переименование категория

In [21]:
cat = pd.Categorical(["a","b","c","a"], categories=["a", "b", "c"])
ser_cat = pd.Series(cat)
ser_cat

0    a
1    b
2    c
3    a
dtype: category
Categories (3, object): ['a', 'b', 'c']

In [30]:
# Переименовать можно либо через замену свойства .categories или через метод .rename_categories

# Переименование категорий (А ТАКЖЕ ЗНАЧЕНИЙ)
# В даном способе возвращается новый объект.
new_cat = ["bronze", "silver", "gold"]
print(ser_cat.cat.set_categories(new_cat, rename=True))
cat = cat.set_categories(new_cat, rename=True)
cat

0    bronze
1    silver
2      gold
3    bronze
dtype: category
Categories (3, object): ['bronze', 'silver', 'gold']


['bronze', 'silver', 'gold', 'bronze']
Categories (3, object): ['bronze', 'silver', 'gold']

#### Чтобы ИЗБЕЖАТЬ переименованя на месте используют метод .rename_categories

In [31]:
print(cat.rename_categories(["x", "y", "z"]))
print(ser_cat.cat.rename_categories(["x", "y", "z"]))

['x', 'y', 'z', 'x']
Categories (3, object): ['x', 'y', 'z']
0    x
1    y
2    z
3    x
dtype: category
Categories (3, object): ['x', 'y', 'z']


In [32]:
# Убедимся, что переименование не затронуло cat
cat, ser_cat

(['bronze', 'silver', 'gold', 'bronze']
 Categories (3, object): ['bronze', 'silver', 'gold'],
 0    a
 1    b
 2    c
 3    a
 dtype: category
 Categories (3, object): ['a', 'b', 'c'])

### Добавлние категорий

In [33]:
# С помощью метода .add_categories можно добавить категории.
with_platinum = metals.add_categories(['platinum'])
with_platinum

['bronze', 'gold', 'silver', 'bronze']
Categories (4, object): ['bronze' < 'silver' < 'gold' < 'platinum']

### Удаление категорий

In [34]:
# Удаление категории можно сделать при помощи метода remove_categories
no_bronze = cat.remove_categories(['bronze'])
no_bronze

[NaN, 'silver', 'gold', NaN]
Categories (2, object): ['gold', 'silver']

### Удаление неиспользуемых катерогий (.remove_unused_categories)

In [35]:
# В with_platinum есть категория platinum, но нет ни одного значения для этой категории
# возвращает новый объект, а старый не трогает.
without_platinum = with_platinum.remove_unused_categories()
without_platinum

['bronze', 'gold', 'silver', 'bronze']
Categories (3, object): ['bronze' < 'silver' < 'gold']

### Установка категорий (.set_categories)

In [36]:
# С помощью метода set_categories МОЖНО СРАЗУ И ДОБИВЛЯТЬ И УДАЛЯТЬ категории
# создаем серию
s = pd.Series(["one", "two", "four", "five"], dtype="category")
print(s, end='\n\n') 
print(s.cat.categories)

0     one
1     two
2    four
3    five
dtype: category
Categories (4, object): ['five', 'four', 'one', 'two']

Index(['five', 'four', 'one', 'two'], dtype='object')


In [38]:
# Следующий код задает категории one и four, а остальные категории удаляются.
# В результате получим NaN для несуществующих категорий.

s = s.cat.set_categories(["one","four"])
s

0     one
1     NaN
2    four
3     NaN
dtype: category
Categories (2, object): ['one', 'four']

### Вычисление описательных статистик для категориальной переменной

In [39]:
# Получение описательной информации о категориальной переменной - .describe
metals.describe()

            counts  freqs
categories               
bronze           2   0.50
silver           1   0.25
gold             1   0.25

In [40]:
# Получить количество для каждой категории можно при помощи метода - .value_counts
metals.value_counts()


bronze    2
silver    1
gold      1
Name: count, dtype: int64

In [44]:
# Вычислим минимум, максимум для категориальной переменной
metals.min(), metals.max()

('bronze', 'gold')

In [45]:
# Вычислим моду для категориальной переменной
pd.Series(metals).mode()

0    bronze
dtype: category
Categories (3, object): ['bronze' < 'silver' < 'gold']

In [48]:
# Обработка школьных ошибок
# 10 учеников со случайными оценками
np.random.seed(280286)
names =  ['Norris', 'Ivan', 'Ruth', 'Lane', 'Skye', 'Sol', 'Dylan', 'Katina', 'Alissa', "Marc"]
grades = np.random.randint(50, 101, len(names))
scores = pd.DataFrame({'Name': names, 'Grade': grades})
scores

     Name  Grade
0  Norris     88
1    Ivan     98
2    Ruth     81
3    Lane     51
4    Skye     79
5     Sol     69
6   Dylan     93
7  Katina     75
8  Alissa     63
9    Marc     89

In [49]:
# Задаем группы и соответствующие оценки

score_bins =  [0, 59, 62, 66, 69, 72, 76, 79, 82, 86, 89, 92, 99, 100]
letter_grades = ['F', 'D-', 'D', 'D+', 'C-', 'C', 'C+', 'B-', 'B', 'B+', 'A-', 'A', 'A+']


In [52]:
# Разбиваем на основе групп и присваиваем буквенные оценки
letter_cats = pd.cut(scores.Grade, bins=score_bins, labels=letter_grades)
scores['Letter'] = letter_cats
scores

     Name  Grade Letter
0  Norris     88     B+
1    Ivan     98      A
2    Ruth     81     B-
3    Lane     51      F
4    Skye     79     C+
5     Sol     69     D+
6   Dylan     93      A
7  Katina     75      C
8  Alissa     63      D
9    Marc     89     B+

In [56]:
# Исследуем интересующую категориальную переменную letter_cats
letter_cats, letter_cats.cat.categories, letter_cats.cat.codes

(0    B+
 1     A
 2    B-
 3     F
 4    C+
 5    D+
 6     A
 7     C
 8     D
 9    B+
 Name: Grade, dtype: category
 Categories (13, object): ['F' < 'D-' < 'D' < 'D+' ... 'B+' < 'A-' < 'A' < 'A+'],
 Index(['F', 'D-', 'D', 'D+', 'C-', 'C', 'C+', 'B-', 'B', 'B+',
        'A-', 'A', 'A+'],
       dtype='object'),
 0     9
 1    11
 2     7
 3     0
 4     6
 5     3
 6    11
 7     5
 8     2
 9     9
 dtype: int8)

In [57]:
# Сколько наблюдений имеет каждая оцинка?
scores.Letter.value_counts()

Letter
B+    2
A     2
F     1
D     1
D+    1
C     1
C+    1
B-    1
D-    0
C-    0
B     0
A-    0
A+    0
Name: count, dtype: int64

In [58]:
# Выберем всех учеников имеющих оценку - А
scores[scores.Letter == 'A']

    Name  Grade Letter
1   Ivan     98      A
6  Dylan     93      A

In [59]:
# Отсортируем по буквенной оценке, а не по числовой
scores.sort_values(by=['Letter'], ascending=True)


     Name  Grade Letter
3    Lane     51      F
8  Alissa     63      D
5     Sol     69     D+
7  Katina     75      C
4    Skye     79     C+
2    Ruth     81     B-
0  Norris     88     B+
9    Marc     89     B+
1    Ivan     98      A
6   Dylan     93      A

In [60]:
cc = pd.cut([0, 1, 1, 6, 11, 12, 13, 15], bins=5)
print(cc, end='\n\n')
print(cc.categories, end='\n\n')
print(cc.codes, end='\n\n')

[(-0.015, 3.0], (-0.015, 3.0], (-0.015, 3.0], (3.0, 6.0], (9.0, 12.0], (9.0, 12.0], (12.0, 15.0], (12.0, 15.0]]
Categories (5, interval[float64, right]): [(-0.015, 3.0] < (3.0, 6.0] < (6.0, 9.0] < (9.0, 12.0] < (12.0, 15.0]]

IntervalIndex([(-0.015, 3.0], (3.0, 6.0], (6.0, 9.0],
               (9.0, 12.0], (12.0, 15.0]],
              dtype='interval[float64, right]')

[0 0 0 1 3 3 4 4]

